# Amalgamation of Data
This notebook demonstrates the amalgamation of Time Series dataset, Weather dataset, Fire Station dataset, and the Fire dataset.

[Global Historical Climatology Network Daily's](https://www.ncdc.noaa.gov/ghcnd-data-access) archived weather data was used from NOAA's National Centers for Environmental Information (NCEI) or formerly known as [NCDC](https://www.ncdc.noaa.gov/). The weather dataset was accessed using [FTP Access](ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/)

Fire dataset was obtained from [Kaggle](https://www.kaggle.com/rtatman/188-million-us-wildfires) containing a spatial SQL database of wildfires that occurred in the United States from 1992 to 2015.

In [0]:
#Import necessary libraries
import requests
import pandas as pd

Download the `Fire` Dataset



In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Google drive link where the dataset is stored.
link = 'https://drive.google.com/open?id=1ApRYxyMqnFKH8XW1GbQ-KZNJdbdeEdV9'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

# Download the fire dataset initially used for exploratory data analysis.
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Fire.csv')


1ApRYxyMqnFKH8XW1GbQ-KZNJdbdeEdV9


Download the `Fire Station` Dataset containing Fire Station's ID and their location.

In [0]:
link = 'https://drive.google.com/open?id=1A7RNorqSglE-G8j5BacVsGYmE6eR4cwx'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

# To download the Fire Station
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ghcnd-stations.csv')

1A7RNorqSglE-G8j5BacVsGYmE6eR4cwx


In [0]:
# Change the range to the year interested to get the weather data.
files = [str(x)+'.csv.gz' for x in range(2015,2016)]
files

['2015.csv.gz']

In [0]:
#fetching data from ftp server for years from 2015 to 2016

from ftplib import FTP
from datetime import datetime

start = datetime.now()
ftp = FTP('ftp.ncdc.noaa.gov')
ftp.login()
ftp.cwd("pub/data/ghcn/daily/by_year/")


# Get All Files
files = [str(x)+'.csv.gz' for x in range(2015,2016)]

# Print out the files
for file in files:
	print("Downloading..." + file)
	ftp.retrbinary("RETR " + file ,open(file, 'wb').write)

ftp.close()

end = datetime.now()
diff = end - start
print('All files downloaded for ' + str(diff.seconds) + 's')

Downloading...2015.csv.gz
All files downloaded for 4s


Install Apache Spark, OpenJDK Development Kit(JDK), and Pyspark

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
!pip install pyspark
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkFiles
import pyspark.sql.functions as func

     |████████████████████████████████| 217.8MB 63kB/s 
     |████████████████████████████████| 204kB 47.2MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=593b7549dd37261f99265b6caa1fb9145e5f327cf823ee53fe61f3088b1d50b8
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

Generating spark contect and sessions to consume our datasets.

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("gfcj") \
        .getOrCreate()

Adding all the files to spark context to spread them into rdd and perform faster execution on huge dataset.

In [0]:
sc=SparkContext.getOrCreate()

for file in files:
  sc.addFile(file)

In [0]:
sqlContext = SQLContext(sc)
sqlContext
# df = sqlContext.read.csv(SparkFiles.get(files), header=True, inferSchema= True)
# df.printSchema()

spark.read.format("csv").option("header", "true").load("*.csv.gz")
df=spark.read.option("header", "false").csv("*.csv.gz")

In [0]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [0]:
df.show(5)

+-----------+--------+----+----+----+----+---+----+
|        _c0|     _c1| _c2| _c3| _c4| _c5|_c6| _c7|
+-----------+--------+----+----+----+----+---+----+
|US1FLSL0019|20150101|PRCP| 173|null|null|  N|null|
|US1TXTV0133|20150101|PRCP| 119|null|null|  N|null|
|USC00178998|20150101|TMAX| -33|null|null|  7|0700|
|USC00178998|20150101|TMIN|-167|null|null|  7|0700|
|USC00178998|20150101|TOBS| -67|null|null|  7|0700|
+-----------+--------+----+----+----+----+---+----+
only showing top 5 rows



Adding Fire-Station Dataset to Spark Context

In [0]:
sc.addFile('ghcnd-stations.csv')

Converting the CSV file into Spark Dataframe

In [0]:
df2=spark.read.option("header", "true").csv('ghcnd-stations.csv')

In [0]:
df2

DataFrame[id: string, lat: string, long: string, _c3: string, _c4: string, _c5: string]

Displaying the first five records

In [0]:
df.show(5)

+-----------+--------+----+----+----+----+---+----+
|        _c0|     _c1| _c2| _c3| _c4| _c5|_c6| _c7|
+-----------+--------+----+----+----+----+---+----+
|US1FLSL0019|20150101|PRCP| 173|null|null|  N|null|
|US1TXTV0133|20150101|PRCP| 119|null|null|  N|null|
|USC00178998|20150101|TMAX| -33|null|null|  7|0700|
|USC00178998|20150101|TMIN|-167|null|null|  7|0700|
|USC00178998|20150101|TOBS| -67|null|null|  7|0700|
+-----------+--------+----+----+----+----+---+----+
only showing top 5 rows



In [0]:
df2.show(5)

+-----------+-------+--------+----+----+----+
|         id|    lat|    long| _c3| _c4| _c5|
+-----------+-------+--------+----+----+----+
|ACW00011604|17.1167|-61.7833|null|null|null|
|ACW00011647|17.1333|-61.7833|null|null|null|
|AE000041196| 25.333|  55.517|null|null|null|
|AEM00041194| 25.255|  55.364|null|null|null|
|AEM00041217| 24.433|  54.651|null|null|null|
+-----------+-------+--------+----+----+----+
only showing top 5 rows



In [0]:
# Renaming the headers
data=df.selectExpr('_c0 as id','_c1 as date','_c2 as type','_c3 as value')

In [0]:
data.show()

+-----------+--------+----+-----+
|         id|    date|type|value|
+-----------+--------+----+-----+
|US1FLSL0019|20150101|PRCP|  173|
|US1TXTV0133|20150101|PRCP|  119|
|USC00178998|20150101|TMAX|  -33|
|USC00178998|20150101|TMIN| -167|
|USC00178998|20150101|TOBS|  -67|
|USC00178998|20150101|PRCP|    0|
|USC00178998|20150101|SNOW|    0|
|USC00178998|20150101|SNWD|    0|
|NOE00133566|20150101|TMAX|   90|
|NOE00133566|20150101|TMIN|   44|
|NOE00133566|20150101|PRCP|   19|
|NOE00133566|20150101|SNWD|   40|
|USC00141761|20150101|TMAX|  -83|
|USC00141761|20150101|TMIN| -183|
|USC00141761|20150101|TOBS| -122|
|USC00141761|20150101|PRCP|    0|
|USC00141761|20150101|SNOW|    0|
|USC00141761|20150101|SNWD|    0|
|USS0018D08S|20150101|TMAX|   -8|
|USS0018D08S|20150101|TMIN| -130|
+-----------+--------+----+-----+
only showing top 20 rows



In [0]:
df2.show(2)

+-----------+-------+--------+----+----+----+
|         id|    lat|    long| _c3| _c4| _c5|
+-----------+-------+--------+----+----+----+
|ACW00011604|17.1167|-61.7833|null|null|null|
|ACW00011647|17.1333|-61.7833|null|null|null|
+-----------+-------+--------+----+----+----+
only showing top 2 rows



Performing an Inner join on ID

In [0]:
final_df = data.join(df2, on=['id'], how='inner')

In [0]:
final_df=final_df.selectExpr('id','lat as LATITUDE','long as LONGITUDE','date','type','value')

In [0]:
final_df.registerTempTable("temperature")

Converting the Timestamp into Julian date format and add Month and Weekday to the dataset.

In [0]:
df=pd.read_csv('Fire.csv')

df['disc_clean_date'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
#Convert containment date from julian to standard date
df['cont_clean_date'] = pd.to_datetime(df['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
# Day of month string
df['discovery_month'] = df['disc_clean_date'].dt.strftime('%b')
#Returns the weekday string
df['discovery_weekday'] = df['disc_clean_date'].dt.strftime('%a')
#Merge discovery date and time 


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df

,Unnamed: 0,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,LOCAL_FIRE_REPORT_ID,LOCAL_INCIDENT_ID,FIRE_CODE,FIRE_NAME,ICS_209_INCIDENT_NUMBER,ICS_209_NAME,MTBS_ID,MTBS_FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape,disc_clean_date,cont_clean_date,discovery_month,discovery_weekday
0,0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,Plumas National Forest,1,PNF-47,BJ8K,FOUNTAIN,NaN,NaN,NaN,NaN,NaN,2005,2453403.5,33,1300.0,9.0,Miscellaneous,2453403.5,33.0,1730.0,0.10,A,40.036944,-121.005833,5.0,USFS,CA,63,63.0,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...,2005-02-02,2005-02-02,Feb,Wed
1,1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,13,13,AAC0,PIGEON,NaN,NaN,NaN,NaN,NaN,2004,2453137.5,133,845.0,1.0,Lightning,2453137.5,133.0,1530.0,0.25,A,38.933056,-120.404444,5.0,USFS,CA,61,61.0,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...,2004-05-12,2004-05-12,May,Wed
2,2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,27,021,A32W,SLACK,NaN,NaN,NaN,NaN,NaN,2004,2453156.5,152,1921.0,5.0,Debris Burning,2453156.5,152.0,2024.0,0.10,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,17.0,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...,2004-05-31,2004-05-31,May,Mon
3,3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,43,6,NaN,DEER,NaN,NaN,NaN,NaN,NaN,2004,2453184.5,180,1600.0,1.0,Lightning,2453189.5,185.0,1400.0,0.10,A,38.559167,-119.913333,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...,2004-06-28,2004-07-03,Jun,Mon
4,4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,44,7,NaN,STEVENOT,NaN,NaN,NaN,NaN,NaN,2004,2453184.5,180,1600.0,1.0,Lightning,2453189.5,185.0,1200.0,0.10,A,38.559167,-119.933056,5.0,USFS,CA,3,3.0,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...,2004-06-28,2004-07-03,Jun,Mon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,1880460,1880461,300348363,2015CAIRS29019636,NONFED,ST-CACDF,ST/C&L,USCASHU,Shasta-Trinity Unit,CASHU,Shasta-Trinity Unit,591814,009371,NaN,ODESSA 2,NaN,NaN,NaN,NaN,NaN,2015,2457291.5,269,1726.0,13.0,Missing/Undefined,2457291.5,269.0,1843.0,0.01,A,40.481637,-122.389375,13.0,STATE OR PRIVATE,CA,NaN,NaN,NaN,b'\x00\x01\xad\x10\x00\x00P\xb8\x1e\x85\xeb\x9...,2015-09-26,2015-09-26,Sep,Sat
1880461,1880461,1880462,300348373,2015CAIRS29217935,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,569419,000366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2457300.5,278,126.0,9.0,Miscellaneous,NaN,NaN,NaN,0.20,A,37.617619,-120.938570,12.0,MUNICIPAL/LOCAL,CA,NaN,NaN,NaN,b'\x00\x01\xad\x10\x00\x00\x00\x80\xbe\x88\x11...,2015-10-05,NaT,Oct,Mon
1880462,1880462,1880463,300348375,2015CAIRS28364460,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,574245,000158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2457144.5,122,2052.0,13.0,Missing/Undefined,NaN,NaN,NaN,0.10,A,37.617619,-120.938570,12.0,MUNICIPAL/LOCAL,CA,NaN,NaN,NaN,b'\x00\x01\xad\x10\x00\x00\x00\x80\xbe\x88\x11...,2015-05-02,NaT,May,Sat
1880463,1880463,1880464,300348377,2015CAIRS29218079,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,570462,000380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015,2457309.5,287,2309.0,13.0,Missing/Undefined,NaN,NaN,NaN,2.00,B,37.672235,-120.898356,12.0,MUNICIPAL/LOCAL,CA,NaN,NaN,NaN,b'\x00\x01\xad\x10\x00\x00x\xba_\xaa~9^\xc0\xb...,2015-10-14,Na

Filter the Fire dataset with records of the year 2015. Change the year for which you are amalgamating.

In [0]:
fire_filter=[2015]
df_2015=df[df.FIRE_YEAR.isin(fire_filter)]

fire_filter

[2015]

Keeping only `Latitude, Longitude, and cont_clean_date` in the Fire dataset.

In [0]:
df_2015=df_2015[['LATITUDE','LONGITUDE','cont_clean_date']]

Dropping all Null values.

In [0]:
df_2015=df_2015.dropna()

In [0]:
df_2015

,LATITUDE,LONGITUDE,cont_clean_date
1789011,45.934167,-113.020833,2015-08-15
1789013,45.727222,-112.943889,2015-07-15
1789015,44.416667,-112.843333,2015-10-26
1789016,46.052222,-113.635556,2015-07-02
1789017,46.234444,-113.301667,2015-05-26
...,...,...,...
1880456,40.019907,-122.391398,2015-06-14
1880457,40.588583,-123.069617,2015-10-01
1880458,40.244833,-123.544167,2015-08-06
1880459,38.415608,-122.660044,2015-05-28


In [0]:
# Converting the timestamp to string.
df_2015['cont_clean_date']=df_2015['cont_clean_date'].astype(str)

In [0]:
# Applying Regex on the cont_clean_date column to remove '-' and adding into the dataset as date column.
df_2015['date']=df_2015['cont_clean_date'].replace('-', '', regex=True).astype(int)

In [0]:
df_2015

,LATITUDE,LONGITUDE,cont_clean_date,date
1789011,45.934167,-113.020833,2015-08-15,20150815
1789013,45.727222,-112.943889,2015-07-15,20150715
1789015,44.416667,-112.843333,2015-10-26,20151026
1789016,46.052222,-113.635556,2015-07-02,20150702
1789017,46.234444,-113.301667,2015-05-26,20150526
...,...,...,...,...
1880456,40.019907,-122.391398,2015-06-14,20150614
1880457,40.588583,-123.069617,2015-10-01,20151001
1880458,40.244833,-123.544167,2015-08-06,20150806
1880459,38.415608,-122.660044,2015-05-28,20150528


In [0]:
df_2015=df_2015[['LATITUDE','LONGITUDE','date']]

In [0]:
# Save the intermediate csv file
df_2015.to_csv(r'fire_2015.csv')

In [0]:
# Add it to the spark context
sc.addFile('fire_2015.csv')

In [0]:
# Convert it into Spark Dataframe
fire=spark.read.option("header", "true").csv('fire_2015.csv')

In [0]:
fire.show(5)

+-------+-----------+-------------+--------+
|    _c0|   LATITUDE|    LONGITUDE|    date|
+-------+-----------+-------------+--------+
|1789011|45.93416667|-113.02083333|20150815|
|1789013|45.72722222|-112.94388889|20150715|
|1789015|44.41666667|-112.84333333|20151026|
|1789016|46.05222222|-113.63555556|20150702|
|1789017|46.23444444|-113.30166667|20150526|
+-------+-----------+-------------+--------+
only showing top 5 rows



In [0]:
# Round the Latitude and Longitude values to scale decimal places.
fire=fire.withColumn("LATITUDE", func.round(fire["LATITUDE"]))
fire=fire.withColumn("LONGITUDE", func.round(fire["LONGITUDE"]))

In [0]:
final_df.show()

+-----------+--------+---------+--------+----+-----+
|         id|LATITUDE|LONGITUDE|    date|type|value|
+-----------+--------+---------+--------+----+-----+
|US1FLSL0019| 27.3237| -80.3111|20150101|PRCP|  173|
|US1TXTV0133| 30.3597| -98.0252|20150101|PRCP|  119|
|USC00178998| 44.6917| -68.3475|20150101|TMAX|  -33|
|USC00178998| 44.6917| -68.3475|20150101|TMIN| -167|
|USC00178998| 44.6917| -68.3475|20150101|TOBS|  -67|
|USC00178998| 44.6917| -68.3475|20150101|PRCP|    0|
|USC00178998| 44.6917| -68.3475|20150101|SNOW|    0|
|USC00178998| 44.6917| -68.3475|20150101|SNWD|    0|
|NOE00133566| 62.1031|   5.5817|20150101|TMAX|   90|
|NOE00133566| 62.1031|   5.5817|20150101|TMIN|   44|
|NOE00133566| 62.1031|   5.5817|20150101|PRCP|   19|
|NOE00133566| 62.1031|   5.5817|20150101|SNWD|   40|
|USC00141761| 39.5592| -97.6697|20150101|TMAX|  -83|
|USC00141761| 39.5592| -97.6697|20150101|TMIN| -183|
|USC00141761| 39.5592| -97.6697|20150101|TOBS| -122|
|USC00141761| 39.5592| -97.6697|20150101|PRCP|

In [0]:
# Round the Latitude and Longitude values to scale decimal places.
final_df=final_df.withColumn("LATITUDE", func.round(final_df["LATITUDE"]))
final_df=final_df.withColumn("LONGITUDE", func.round(final_df["LONGITUDE"]))

In [0]:
final_df.show()

+-----------+--------+---------+--------+----+-----+
|         id|LATITUDE|LONGITUDE|    date|type|value|
+-----------+--------+---------+--------+----+-----+
|US1FLSL0019|    27.0|    -80.0|20150101|PRCP|  173|
|US1TXTV0133|    30.0|    -98.0|20150101|PRCP|  119|
|USC00178998|    45.0|    -68.0|20150101|TMAX|  -33|
|USC00178998|    45.0|    -68.0|20150101|TMIN| -167|
|USC00178998|    45.0|    -68.0|20150101|TOBS|  -67|
|USC00178998|    45.0|    -68.0|20150101|PRCP|    0|
|USC00178998|    45.0|    -68.0|20150101|SNOW|    0|
|USC00178998|    45.0|    -68.0|20150101|SNWD|    0|
|NOE00133566|    62.0|      6.0|20150101|TMAX|   90|
|NOE00133566|    62.0|      6.0|20150101|TMIN|   44|
|NOE00133566|    62.0|      6.0|20150101|PRCP|   19|
|NOE00133566|    62.0|      6.0|20150101|SNWD|   40|
|USC00141761|    40.0|    -98.0|20150101|TMAX|  -83|
|USC00141761|    40.0|    -98.0|20150101|TMIN| -183|
|USC00141761|    40.0|    -98.0|20150101|TOBS| -122|
|USC00141761|    40.0|    -98.0|20150101|PRCP|

Merge the Fire Dataset with the Spark Dataframe

In [0]:
complete = fire.join(final_df, on=['LATITUDE','LONGITUDE','date'], how='inner')

In [0]:
complete.show(1)

+--------+---------+--------+-------+-----------+----+-----+
|LATITUDE|LONGITUDE|    date|    _c0|         id|type|value|
+--------+---------+--------+-------+-----------+----+-----+
|    34.0|   -112.0|20150101|1852766|US1AZMR0026|PRCP|   13|
+--------+---------+--------+-------+-----------+----+-----+
only showing top 1 row



In [0]:
# Store the resulting Spark Dataframe into the local machine in order to convert the csv file into pandas dataframe.
complete.write.csv('2015Py.csv',header='true')

In [0]:
# Read the csv file using pandas
csv = pd.read_csv('completePandas.csv')

In [0]:
completepandas_df=csv

In [0]:
# Displays the size of the pandas dataframe
completepandas_df.shape

(6255725, 7)

In [0]:
completepandas_df.head

<bound method NDFrame.head of          LATITUDE  LONGITUDE      date      _c0           id  type  value
0            34.0     -112.0  20150101  1852766  US1AZMR0026  PRCP     13
1            34.0     -112.0  20150101  1852762  US1AZMR0026  PRCP     13
2            34.0     -112.0  20150101  1852761  US1AZMR0026  PRCP     13
3            34.0     -112.0  20150101  1852760  US1AZMR0026  PRCP     13
4            36.0      -78.0  20150101  1857745  US1NCJH0009  PRCP      0
...           ...        ...       ...      ...          ...   ...    ...
6255720      34.0     -112.0  20151231  1856828  US1AZMR0164  SNOW      0
6255721      34.0     -110.0  20151231  1796878  US1AZNV0009  PRCP      0
6255722      34.0     -110.0  20151231  1796878  US1AZNV0009  SNOW      0
6255723      40.0      -74.0  20151231  1813028  US1NJMN0010  PRCP     69
6255724      40.0      -74.0  20151231  1813027  US1NJMN0010  PRCP     69

[6255725 rows x 7 columns]>

In [0]:
completepandas_df

,LATITUDE,LONGITUDE,date,_c0,id,type,value
0,34.0,-112.0,20150101,1852766,US1AZMR0026,PRCP,13
1,34.0,-112.0,20150101,1852762,US1AZMR0026,PRCP,13
2,34.0,-112.0,20150101,1852761,US1AZMR0026,PRCP,13
3,34.0,-112.0,20150101,1852760,US1AZMR0026,PRCP,13
4,36.0,-78.0,20150101,1857745,US1NCJH0009,PRCP,0
...,...,...,...,...,...,...,...
6255720,34.0,-112.0,20151231,1856828,US1AZMR0164,SNOW,0
6255721,34.0,-110.0,20151231,1796878,US1AZNV0009,PRCP,0
6255722,34.0,-110.0,20151231,1796878,US1AZNV0009,SNOW,0
6255723,40.0,-74.0,20151231,1813028,US1NJMN0010,PRCP,69


Store only `'LATITUDE', 'LONGITUDE', 'date', 'type', and 'value'` columns into the pandas dataframe.

In [0]:
completepandas_df=completepandas_df[['LATITUDE','LONGITUDE','date','type','value']]

In [0]:
temp1=completepandas_df[completepandas_df.type=='TMAX']
temp2=completepandas_df[completepandas_df.type=='PRCP']
temp3=completepandas_df[completepandas_df.type=='TOBS']
temp4=completepandas_df[completepandas_df.type=='SNOW']

Fetch the `Maximum Temperature, Precipitation, Snow level, and Time of Observation changes` from the dataframe and add an additional column with those matching.

In [0]:
temp1=temp1[['LATITUDE','LONGITUDE','date','value']]
temp1.rename(columns={'value':'TMAX'}, inplace=True)
temp1

,LATITUDE,LONGITUDE,date,TMAX
10,40.0,-96.0,20150101,-72
14,38.0,-120.0,20150101,72
22,41.0,-74.0,20150101,33
23,41.0,-74.0,20150101,33
24,41.0,-74.0,20150101,33
...,...,...,...,...
6255647,34.0,-112.0,20151231,156
6255648,34.0,-112.0,20151231,156
6255683,49.0,-120.0,20151231,-75
6255691,40.0,-102.0,20151231,-6


In [0]:
temp2=temp2[['LATITUDE','LONGITUDE','date','value']]
temp2.rename(columns={'value':'PRCP'}, inplace=True)
temp2

,LATITUDE,LONGITUDE,date,PRCP
0,34.0,-112.0,20150101,13
1,34.0,-112.0,20150101,13
2,34.0,-112.0,20150101,13
3,34.0,-112.0,20150101,13
4,36.0,-78.0,20150101,0
...,...,...,...,...
6255717,34.0,-112.0,20151231,0
6255718,34.0,-112.0,20151231,0
6255721,34.0,-110.0,20151231,0
6255723,40.0,-74.0,20151231,69


In [0]:
temp3=temp3[['LATITUDE','LONGITUDE','date','value']]
temp3.rename(columns={'value':'TOBS'}, inplace=True)
temp3

,LATITUDE,LONGITUDE,date,TOBS
12,40.0,-96.0,20150101,-128
96,38.0,-93.0,20150101,-50
113,41.0,-72.0,20150101,44
287,36.0,-78.0,20150101,-6
297,39.0,-94.0,20150101,-83
...,...,...,...,...
6255585,49.0,-119.0,20151231,-139
6255586,49.0,-119.0,20151231,-139
6255587,49.0,-119.0,20151231,-139
6255626,31.0,-83.0,20151231,200


In [0]:
temp4=temp4[['LATITUDE','LONGITUDE','date','value']]
temp4.rename(columns={'value':'SNOW'}, inplace=True)
temp4

,LATITUDE,LONGITUDE,date,SNOW
5,36.0,-78.0,20150101,0
7,41.0,-76.0,20150101,0
9,38.0,-93.0,20150101,0
20,42.0,-89.0,20150101,0
45,39.0,-94.0,20150101,0
...,...,...,...,...
6255715,33.0,-112.0,20151231,0
6255716,33.0,-112.0,20151231,0
6255719,34.0,-112.0,20151231,0
6255720,34.0,-112.0,20151231,0


In [0]:
# temp1.astype({'TMAX': 'int32'}).dtypes
temp1['TMAX']=pd.to_numeric(temp1['TMAX'])
temp2['PRCP']=pd.to_numeric(temp2['PRCP'])
temp3['TOBS']=pd.to_numeric(temp3['TOBS'])
temp4['SNOW']=pd.to_numeric(temp4['SNOW'])

In [0]:
temp2

,LATITUDE,LONGITUDE,date,PRCP
0,34.0,-112.0,20150101,13
1,34.0,-112.0,20150101,13
2,34.0,-112.0,20150101,13
3,34.0,-112.0,20150101,13
4,36.0,-78.0,20150101,0
...,...,...,...,...
6255717,34.0,-112.0,20151231,0
6255718,34.0,-112.0,20151231,0
6255721,34.0,-110.0,20151231,0
6255723,40.0,-74.0,20151231,69


In [0]:
temp1=temp1.groupby(by=['LATITUDE','LONGITUDE','date']).mean().reset_index()
temp2=temp2.groupby(by=['LATITUDE','LONGITUDE','date']).mean().reset_index()
temp3=temp3.groupby(by=['LATITUDE','LONGITUDE','date']).mean().reset_index()
temp4=temp4.groupby(by=['LATITUDE','LONGITUDE','date']).mean().reset_index()

Merge the resulting dataframes containing Maximum Temperature, Precipitation, Snow level, and Time of Observation changes into `temp`.

In [0]:
x=pd.merge(temp1, temp2, on=['LATITUDE','LONGITUDE','date'], how='inner')
y=pd.merge(temp3, temp4, on=['LATITUDE','LONGITUDE','date'], how='inner')
temp=pd.merge(x, y, on=['LATITUDE','LONGITUDE','date'], how='inner')

In [0]:
temp

,LATITUDE,LONGITUDE,date,TMAX,PRCP,TOBS,SNOW
0,18.0,-67.0,20150313,289.500000,5.657143,211.285714,0.0
1,18.0,-67.0,20150317,286.222222,37.000000,202.125000,0.0
2,18.0,-67.0,20150319,290.222222,14.000000,219.250000,0.0
3,18.0,-67.0,20150501,311.666667,0.000000,235.111111,0.0
4,18.0,-67.0,20150623,311.300000,134.843750,246.888889,0.0
...,...,...,...,...,...,...,...
31978,65.0,-141.0,20150618,245.000000,0.000000,102.500000,0.0
31979,65.0,-141.0,20150621,269.750000,1.000000,140.500000,0.0
31980,65.0,-141.0,20150813,164.750000,19.333333,79.000000,0.0
31981,65.0,-141.0,20151005,60.000000,0.000000,-4.500000,0.0


Converting the Timestamp into Julian date format and add Month and Weekday to the dataset.

In [0]:
fire=pd.read_csv('Fire.csv')
df=fire

df['disc_clean_date'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
#Convert containment date from julian to standard date
df['cont_clean_date'] = pd.to_datetime(df['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
# Day of month string
df['discovery_month'] = df['disc_clean_date'].dt.strftime('%b')
#Returns the weekday string
df['discovery_weekday'] = df['disc_clean_date'].dt.strftime('%a')
df['cont_clean_date']=df['cont_clean_date'].astype(str)
df=df[df.cont_clean_date!='NaT']
df['date']=df['cont_clean_date'].replace('-', '', regex=True).astype(int)
df=df[['LATITUDE','LONGITUDE','STAT_CAUSE_CODE','DISCOVERY_DOY','date','FIRE_SIZE']]
fire=df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,11,12,13,14,15,16,17,18,19,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [0]:
# Round the Latitude and Longitude values to scale decimal places.
fire['LATITUDE']=fire['LATITUDE'].round()
fire['LONGITUDE']=fire['LONGITUDE'].round()


In [0]:
# Converting the date to Integer Datatype
fire['date']=fire['date'].astype(int)
temp['date']=temp['date'].astype(int)

In [0]:
fire

,LATITUDE,LONGITUDE,STAT_CAUSE_CODE,DISCOVERY_DOY,date,FIRE_SIZE
0,40.0,-121.0,9.0,33,20050202,0.10
1,39.0,-120.0,1.0,133,20040512,0.25
2,39.0,-121.0,5.0,152,20040531,0.10
3,39.0,-120.0,1.0,180,20040703,0.10
4,39.0,-120.0,1.0,180,20040703,0.10
...,...,...,...,...,...,...
1880456,40.0,-122.0,13.0,165,20150614,2.22
1880457,41.0,-123.0,7.0,273,20151001,1.00
1880458,40.0,-124.0,1.0,213,20150806,4.00
1880459,38.0,-123.0,9.0,148,20150528,0.50


In [0]:
# Perform the inner join temp dataframe and fire
finalFinal = pd.merge(temp, fire, on=['LATITUDE','LONGITUDE','date'], how='inner')

In [0]:
finalFinal

,LATITUDE,LONGITUDE,date,TMAX,PRCP,TOBS,SNOW,STAT_CAUSE_CODE,DISCOVERY_DOY,FIRE_SIZE
0,18.0,-67.0,20150313,289.500000,5.657143,211.285714,0.0,7.0,71,1.10
1,18.0,-67.0,20150313,289.500000,5.657143,211.285714,0.0,7.0,71,1.80
2,18.0,-67.0,20150313,289.500000,5.657143,211.285714,0.0,7.0,71,0.75
3,18.0,-67.0,20150317,286.222222,37.000000,202.125000,0.0,7.0,76,0.50
4,18.0,-67.0,20150319,290.222222,14.000000,219.250000,0.0,7.0,78,0.50
...,...,...,...,...,...,...,...,...,...,...
60354,65.0,-141.0,20150618,245.000000,0.000000,102.500000,0.0,1.0,144,5.00
60355,65.0,-141.0,20150621,269.750000,1.000000,140.500000,0.0,1.0,148,0.30
60356,65.0,-141.0,20150813,164.750000,19.333333,79.000000,0.0,1.0,144,2902.30
60357,65.0,-141.0,20151005,60.000000,0.000000,-4.500000,0.0,1.0,128,56.00


Save the Final CSV file for that particular year. This process is repeated for all the 21 years.

In [0]:
finalFinal.to_csv('2015.csv')

The below code shows how to aggregate individual 21 years of dataset into one final dataset.

In [0]:
import os
import glob
import pandas as pd

In [0]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [0]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv("final.csv", index=False, encoding='utf-8-sig')